# Correlation and Convolution
Convolution is a filtering operation that expresses the amount of overlap of one function as it is shifted over another function.
$$f \otimes h = \sum_k\sum_l f(k, l)h(k, l)$$

Correlation compares the similarity of two sets of data
$$f * h = \sum_k\sum_l f(k, l)h(-k, -l)$$

# Gaussian
### Gaussian Filter
Used for blurring images
- 1D: $g(x)=e^\frac{-x^2}{2\sigma ^2}$
- 2D: $g(x, y)=e^\frac{-(x^2 + y^2)}{2\sigma ^2}$

### Laplacian of Gaussian
Detects edges through zero crossings
- 1D: $LoG(x)=-\frac{1}{\pi\sigma ^4}\Big[ 1-\frac{x^2}{2\sigma ^2}\Big]e^\frac{-x^2}{2\sigma ^2}$
- 2D: $LoG(x, y)=-\frac{1}{\pi\sigma ^4}\Big[ 1-\frac{x^2 + y^2}{2\sigma ^2}\Big]e^\frac{-(x^2 + y^2)}{2\sigma ^2}$

In [ ]:
import torch
import torch.nn.functional as F
from torchvision import transforms
from PIL import Image
import matplotlib.pyplot as plt

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
def gaussian_blur(image_tensor: torch.Tensor, kernel_size=5, sigma=1.0):
    ax = torch.arange(-kernel_size // 2 + 1.0, kernel_size // 2 + 1.0, device=device, dtype=torch.float)
    xx, yy = torch.meshgrid(ax, ax, indexing=None)

    # Build gaussian kernel using formula
    gaussian_kernel = torch.exp(-0.5 * (xx ** 2 + yy ** 2) / sigma ** 2)
    gaussian_kernel /= gaussian_kernel.sum()

    # Add batch and channel dimensions
    image_tensor = image_tensor.unsqueeze(0)
    gaussian_kernel = gaussian_kernel.view(1, 1, kernel_size, kernel_size)
    blurred_image_tensor = F.conv2d(image_tensor, gaussian_kernel, padding=kernel_size // 2)
    return blurred_image_tensor.squeeze(0)

def gaussian_noise(image_tensor: torch.Tensor, mean=0.0, std=0.1):
    noise = torch.randn(image_tensor.size(), device=device) * std + mean
    noisy_image = image_tensor + noise
    
    return torch.clamp(noisy_image, 0, 1)

def laplacian_of_gaussian(image_tensor: torch.Tensor, kernel_size=5, sigma=1.0):
    # Get x and y
    ax = torch.arange(-kernel_size // 2 + 1.0, kernel_size // 2 + 1.0, device=device, dtype=torch.float)
    xx, yy = torch.meshgrid(ax, ax, indexing=None)

    # Build gaussian kernel using formula
    gaussian_kernel = torch.exp(-0.5 * (xx**2 + yy**2) / sigma**2)
    gaussian_kernel /= gaussian_kernel.sum()
    
    # Apply laplacian operator to gaussian kernel
    log_kernel = -1 // (torch.pi * sigma ** 4) * (1 - (xx ** 2 + yy ** 2) / (2 * sigma ** 2)) * gaussian_kernel
    log_kernel -= log_kernel.mean()

    # Convolve image
    log_kernel = log_kernel.view(1, 1, kernel_size, kernel_size)
    log_image_tensor = F.conv2d(image_tensor, log_kernel, padding=kernel_size//2)
    
    return log_image_tensor.squeeze()

In [ ]:
toTensor = transforms.ToTensor()
toImage = transforms.ToPILImage()
toGrayscale = transforms.Grayscale()

In [ ]:
image = Image.open("img1.png")

# Create tensors for original image, blurred image, and noisy image
image_tensor = toGrayscale(toTensor(image)[0:3]).to(device)
blurred_image_tensor = gaussian_blur(image_tensor)
noisy_image_tensor = gaussian_noise(image_tensor)
log_image_tensor = laplacian_of_gaussian(image_tensor)

# Generate images from tensors
image = toImage(image_tensor)
blurred_image = toImage(blurred_image_tensor)
noisy_image = toImage(noisy_image_tensor)
log_image = toImage(log_image_tensor)

### Display images

In [ ]:
plt.figure(figsize=(12, 5))

plt.subplot(1, 4, 1)
plt.imshow(image, 'gray')
plt.axis('off')
plt.title('Original')

plt.subplot(1, 4, 2)
plt.imshow(blurred_image, 'gray')
plt.axis('off')
plt.title('Blurred')

plt.subplot(1, 4, 3)
plt.imshow(noisy_image, 'gray')
plt.axis('off')
plt.title('Noisy')

plt.subplot(1, 4, 4)
plt.imshow(log_image, 'gray')
plt.axis('off')
plt.title('Laplacian of Gaussian')

plt.show()